In [1]:
import os
import glob
import pandas as pd
import numpy as np
import scipy.stats
import plotly.express as px

In [17]:
file_path = r'C:/Users/Ryan/Documents/BootCamp/Final_Project/Disney_wait_times'

In [32]:
# Find all CSV files containing data from file_path and add to a list

wait_data = os.path.join(file_path, 'Wait_Time_Day_Avg/aggregated_data.csv')

In [33]:
# Concatenate CSV files from list into a single dataframe

wait_time_df = pd.read_csv(wait_data)

In [34]:
wait_time_df.head()

,date,ride_name,SACTMIN,SPOSTMIN
0,1/1/2015,7_dwarfs_train,54.500000,86.429630
1,1/1/2016,7_dwarfs_train,20.333333,69.140127
2,1/1/2017,7_dwarfs_train,46.500000,77.615385
3,1/1/2018,7_dwarfs_train,34.500000,37.140940
4,1/1/2019,7_dwarfs_train,52.000000,0.370629


In [35]:
wait_time_df.shape

(18166, 4)

In [ ]:
wait_time_df['SACTMIN'].describe()

In [36]:
wait_time_df.isnull().sum()

date            0
ride_name       0
SACTMIN      2055
SPOSTMIN        3
dtype: int64

In [37]:
wait_time_df.dtypes

date          object
ride_name     object
SACTMIN      float64
SPOSTMIN     float64
dtype: object

In [38]:
wait_time_df = wait_time_df.rename(columns={'SACTMIN': 'sactmin', 'SPOSTMIN': 'spostmin'})

In [40]:
wait_time_actuals = wait_time_df.dropna(subset=['sactmin'])
wait_time_actuals.head()

,date,ride_name,sactmin,spostmin
0,1/1/2015,7_dwarfs_train,54.500000,86.429630
1,1/1/2016,7_dwarfs_train,20.333333,69.140127
2,1/1/2017,7_dwarfs_train,46.500000,77.615385
3,1/1/2018,7_dwarfs_train,34.500000,37.140940
4,1/1/2019,7_dwarfs_train,52.000000,0.370629


In [41]:
wait_time_actuals.count()

date         16111
ride_name    16111
sactmin      16111
spostmin     16108
dtype: int64

In [42]:
# Dropping column in actuals dataframe that is not needed

wait_time_actuals = wait_time_actuals.drop(columns='spostmin')

In [43]:
wait_time_actuals['sactmin'].describe()

count    16111.000000
mean        22.661698
std         15.313893
min          0.000000
25%         12.500000
50%         19.500000
75%         29.000000
max        176.000000
Name: sactmin, dtype: float64

In [44]:
# Converting datetime column to datetime data type

wait_time_actuals['date'] = pd.to_datetime(wait_time_actuals['date'])

In [45]:
wait_time_actuals.dtypes

date         datetime64[ns]
ride_name            object
sactmin             float64
dtype: object

In [47]:
# Splitting year and month off of datetime for further EDA

wait_time_actuals['year'] = pd.DatetimeIndex(wait_time_actuals['date']).year
wait_time_actuals['month'] = pd.DatetimeIndex(wait_time_actuals['date']).month

In [48]:
wait_time_actuals.head()

,date,ride_name,sactmin,year,month
0,2015-01-01,7_dwarfs_train,54.500000,2015,1
1,2016-01-01,7_dwarfs_train,20.333333,2016,1
2,2017-01-01,7_dwarfs_train,46.500000,2017,1
3,2018-01-01,7_dwarfs_train,34.500000,2018,1
4,2019-01-01,7_dwarfs_train,52.000000,2019,1


In [49]:
wait_time_actuals.shape

(16111, 5)

In [50]:
wait_time_actuals.groupby('year')['sactmin'].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2015,2507.0,20.015292,13.273558,0.0,11.000000,17.000000,25.707143,112.0
2016,2464.0,20.044609,13.612648,0.0,11.000000,17.138889,25.500000,121.0
2017,2395.0,22.529847,15.713600,0.0,12.333333,19.000000,28.600000,141.0
2018,2476.0,23.529863,16.134983,0.0,13.000000,20.000000,29.270833,146.0
2019,2436.0,24.842270,17.564388,0.0,13.750000,21.000000,30.500000,161.0
2020,981.0,27.058093,19.105794,0.0,15.000000,23.000000,33.000000,176.0
2021,2852.0,23.231304,12.810824,1.0,13.000000,21.837719,31.116667,100.5


In [51]:
wait_time_actuals.groupby('ride_name')['sactmin'].describe()

,count,mean,std,min,25%,50%,75%,max
ride_name,,,,,,,,
7_dwarfs_train,2029.0,38.030281,20.791779,0.0,24.875000,34.000000,46.100000,176.0
big_thunder_mtn,2107.0,20.393605,11.037504,0.0,13.000000,18.583333,25.500000,117.0
haunted_mansion,2216.0,17.290049,9.495683,0.0,10.618750,15.500000,22.000000,75.0
it_s_a_small_world,2076.0,13.918996,8.865645,0.0,8.000000,12.000000,17.892857,62.0
peter_pan_s_flight,1735.0,24.965081,14.356658,0.0,16.000000,22.333333,30.171429,111.0
pirates_of_caribbean,2118.0,18.826708,9.572459,0.0,12.000000,17.645833,24.333333,64.0
space_mountain,2011.0,25.139874,15.969785,0.0,14.464286,22.000000,31.800000,132.0
splash_mountain,1819.0,24.196590,15.702603,0.0,13.000000,21.250000,32.535714,120.0


In [53]:
# Distribution of actual wait time data from 2015 to 2021 after dropping three impossible values above

fig = px.histogram(wait_time_actuals, x='sactmin', title='Distribution of Wait Times for Eight Magic Kingdom Rides(2015-2021)')
fig.show()

In [54]:
normal_test = scipy.stats.shapiro
norm_test_act = wait_time_actuals.groupby('year')['sactmin'].apply(normal_test)
norm_test_act2 = norm_test_act.apply(pd.Series, index=['stat', 'p'])
norm_test_act2['normal'] = np.where(norm_test_act2['p']<0.05, 'not normal', 'normal')

norm_test_act2


,stat,p,normal
year,,,
2015,0.879682,2.317117e-40,not normal
2016,0.866782,1.331374e-41,not normal
2017,0.846681,2.886675e-43,not normal
2018,0.817176,0.000000e+00,not normal
2019,0.833739,8.407791e-45,not normal
2020,0.823099,1.573333e-31,not normal
2021,0.954790,7.257688e-29,not normal


In [55]:
Q1 = wait_time_actuals['sactmin'].quantile(0.25)
Q3 = wait_time_actuals['sactmin'].quantile(0.75)
IQR = Q3 - Q1

In [56]:
outliers = wait_time_actuals['sactmin'] > (Q3 + 1.5 * IQR)
outliers.value_counts()


False    15442
True       669
Name: sactmin, dtype: int64

In [57]:
wait_time_actuals = wait_time_actuals.sort_values(by='date')
wait_time_actuals.head(30)

,date,ride_name,sactmin,year,month
0,2015-01-01,7_dwarfs_train,54.500000,2015,1
2334,2015-01-01,big_thunder_mtn,37.000000,2015,1
4556,2015-01-01,haunted_mansion,27.500000,2015,1
6885,2015-01-01,it_s_a_small_world,15.000000,2015,1
16022,2015-01-01,splash_mountain,11.500000,2015,1
9186,2015-01-01,peter_pan_s_flight,33.000000,2015,1
11486,2015-01-01,pirates_of_caribbean,30.000000,2015,1
13688,2015-01-01,space_mountain,6.000000,2015,1
77,2015-01-02,7_dwarfs_train,65.750000,2015,1
16064,2015-01-02,splash_mountain,1.000000,2015,1


In [59]:
wait_time_actuals['id'] = wait_time_actuals.groupby('date', sort=False).ngroup().apply('{:004}'.format)
wait_time_actuals.head(10)

,date,ride_name,sactmin,year,month,id
0,2015-01-01,7_dwarfs_train,54.50,2015,1,0000
2334,2015-01-01,big_thunder_mtn,37.00,2015,1,0000
4556,2015-01-01,haunted_mansion,27.50,2015,1,0000
6885,2015-01-01,it_s_a_small_world,15.00,2015,1,0000
16022,2015-01-01,splash_mountain,11.50,2015,1,0000
9186,2015-01-01,peter_pan_s_flight,33.00,2015,1,0000
11486,2015-01-01,pirates_of_caribbean,30.00,2015,1,0000
13688,2015-01-01,space_mountain,6.00,2015,1,0000
77,2015-01-02,7_dwarfs_train,65.75,2015,1,0001
16064,2015-01-02,splash_mountain,1.00,2015,1,0001


In [61]:
wait_time_actuals.tail(10)

,date,ride_name,sactmin,year,month,id
16646,2021-12-27,splash_mountain,32.000000,2021,12,2328
14441,2021-12-27,space_mountain,55.000000,2021,12,2328
3038,2021-12-27,big_thunder_mtn,22.666667,2021,12,2328
7644,2021-12-28,it_s_a_small_world,14.000000,2021,12,2329
3045,2021-12-28,big_thunder_mtn,20.833333,2021,12,2329
5311,2021-12-28,haunted_mansion,11.600000,2021,12,2329
16653,2021-12-28,splash_mountain,21.500000,2021,12,2329
12245,2021-12-28,pirates_of_caribbean,21.000000,2021,12,2329
760,2021-12-28,7_dwarfs_train,29.800000,2021,12,2329
14448,2021-12-28,space_mountain,43.500000,2021,12,2329


In [62]:
wait_time_actuals = wait_time_actuals[['id', 'date', 'ride_name', 'sactmin']]
wait_time_actuals.head()

,id,date,ride_name,sactmin
0,0000,2015-01-01,7_dwarfs_train,54.5
2334,0000,2015-01-01,big_thunder_mtn,37.0
4556,0000,2015-01-01,haunted_mansion,27.5
6885,0000,2015-01-01,it_s_a_small_world,15.0
16022,0000,2015-01-01,splash_mountain,11.5


In [63]:
export_path = r'C:\Users\Ryan\Documents\BootCamp\Final_Project\Disney_wait_times\clean_wait_times.csv'

In [64]:
wait_time_actuals.to_csv(export_path, sep=',', index=False)